In [10]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import StandardScaler

def build_lookalike_model(transactions_path, customers_path, products_path, output_path):


      customers = pd.read_csv('Customers.csv')
      products = pd.read_csv('Products.csv')
      transactions = pd.read_csv('Transactions.csv')


      data = transactions.merge(customers, on='CustomerID').merge(products, on='ProductID')


      customer_data = data.groupby('CustomerID').agg({
        'TotalValue': 'sum',
        'Quantity': 'sum',
        'Price_y': 'mean'
    }).reset_index()


      scaler = StandardScaler()
      normalized_data = scaler.fit_transform(customer_data[['TotalValue', 'Quantity', 'Price_y']])


      similarity_matrix = cosine_similarity(normalized_data)


      top_lookalikes = {}
      for i, customer_id in enumerate(customer_data['CustomerID'][:20]):
        similar_customers = sorted(
            list(enumerate(similarity_matrix[i])),
            key=lambda x: x[1],
            reverse=True
        )[1:4]
        top_lookalikes[customer_id] = [
            (customer_data['CustomerID'][j], round(score, 2)) for j, score in similar_customers
        ]

      lookalike_df = pd.DataFrame({
        'CustomerID': list(top_lookalikes.keys()),
        'Lookalikes': list(top_lookalikes.values())
    })
      lookalike_df.to_csv(output_path, index=False)
      print(f"Lookalike recommendations saved to {output_path}")


